### Pairs

#### Task description
Find all the pairs of two consequent words where the first word is “narodnaya”. For each pair, count the number of occurrences in the Wikipedia dump. Print all the pairs with their count in a lexicographical order. Output format is:  
“word_pair < tab > count”

In [1]:
import re

from pyspark import SparkConf, SparkContext
from operator import add

sc = SparkContext(conf=SparkConf().setAppName('Spark pairs example').setMaster('local'))

def get_pairs(line):
    word = 'narodnaya'
    try:
        article_id, text = unicode(line.rstrip()).split('\t', 1)
        text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
        words = re.split("\W*\s+\W*", text, flags=re.UNICODE)

        result = []
        for idx, w in enumerate(words):
            w = w.lower()
            if w.lower() == word and idx + 1 != len(words):
                result.append(w + '_' + words[idx+1].lower())
              
        return result
    except ValueError as e:
        return []
    
wiki = sc.textFile("/data/wiki/en_articles_part/articles-part", 16) \
                  .flatMap(get_pairs) \
                  .map(lambda x: (x, 1)) \
                  .reduceByKey(add) \
                  .sortByKey()
    
output = wiki.collect()

for w in output:
    print "%s %d" % w

narodnaya_gazeta 1
narodnaya_volya 9
